# Creating Molecular Graphs in Graphein

[Graphein](https://github.com/a-r-j/graphein) provides a set of tools for working with molecular graphs. Under the hood, we wrap [RDKit](https://www.rdkit.org/) functions for the graph construction. While the main format used is a [NetworkX](https://networkx.org/) `nx.Graph`, each graph, node & edge has its underlying [RDKit object](https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html) stored as metadata so RDKit methods can still be used.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/molecule_tutorial.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/a-r-j/graphein/blob/master/notebooks/molecule_tutorial.ipynb)

In [ ]:
# Install Graphein if necessary
# !pip install graphein[extras]

In [ ]:
import networkx as nx
import logging
from rich import inspect, print
logging.getLogger("matplotlib").setLevel(logging.WARNING)

import graphein.molecule as gm

## Config
Similar to [Protein Graph Construction](https://graphein.ai/notebooks/residue_graphs.html), configuration of small molecule graphs is controlled by a [`MoleculeGraphConfig`]() object.

In [ ]:
config = gm.MoleculeGraphConfig()
inspect(config)

## Creating Graphs

### Creating a Molecular Graph from SMILES

Let's take a look at a simple molecule, acetylsalicylic acid (or aspirin as it's more commonly known).

In [ ]:
graph = gm.construct_graph(smiles="CC(=O)OC1=CC=CC=C1C(=O)O", config=config)
print(graph)
nx.draw(graph)
graph.graph["rdmol"]

Molecule features are stored as dictionaries on the graph, nodes and edges.

In [ ]:
# Node metadata
for n, d in graph.nodes(data=True):
    print(d)

In [ ]:
# Edge metadata
for u, v, d in graph.edges(data=True):
    print(d)

### Creating a Molecular Graph from an SDF File

As SDFs & Mol2 files provide coordinates, we can draw the graph in 3D.

In [ ]:
graph = gm.construct_graph(path="../tests/molecule/test_data/long_test.sdf", config=config)
print(graph)
graph.graph["rdmol"]

In [ ]:
gm.plotly_molecular_graph(graph)

### Creating a Molecular Graph from a Mol2 File

In [ ]:
graph = gm.construct_graph(path="../tests/molecule/test_data/short_test.mol2", config=config)
print(graph)
graph.graph["rdmol"]

In [ ]:
gm.plotly_molecular_graph(graph)

## Adding Features
Graphein can add lots of features to graph nodes and edges from RDKit

### Node Features




In [ ]:
from functools import partial

config = gm.MoleculeGraphConfig(
    node_metadata_functions=[
        gm.atom_type_one_hot,
        gm.atomic_mass,
        gm.degree,
        gm.total_degree,
        gm.total_valence,
        gm.explicit_valence,
        gm.implicit_valence,
        gm.num_explicit_h,
        gm.num_implicit_h,
        gm.total_num_h,
        gm.num_radical_electrons,
        gm.formal_charge,
        gm.hybridization,
        gm.is_aromatic,
        gm.is_isotope,
        gm.is_ring,
        gm.chiral_tag,
        partial(gm.is_ring_size, ring_size=5),
        partial(gm.is_ring_size, ring_size=7)
    ]
)
graph = gm.construct_graph(path="../tests/molecule/test_data/short_test.mol2", config=config)

for n, d in graph.nodes(data=True):
    print(d)

### Edge Features

In [ ]:
config = gm.MoleculeGraphConfig(
    edge_metadata_functions=[
        gm.add_bond_type,
        gm.bond_is_aromatic,
        gm.bond_is_conjugated,
        gm.bond_is_in_ring,
        gm.bond_stereo,
        partial(gm.bond_is_in_ring_size, ring_size=5),
        partial(gm.bond_is_in_ring_size, ring_size=7)
    ]
)
graph = gm.construct_graph(path="../tests/molecule/test_data/short_test.mol2", config=config)

for u, v, d in graph.edges(data=True):
    print(d)

### Graph Features
Global descriptors of the molecule computed by RDKit can be added to the graph too!

In [ ]:
config = gm.MoleculeGraphConfig(
    graph_metadata_functions=[
        gm.mol_descriptors
    ]
)
graph = gm.construct_graph(path="../tests/molecule/test_data/short_test.mol2", config=config)

print(graph.graph)